In [1]:
# Installation of tangelo if not already installed.
try:
    import tangelo
except ModuleNotFoundError:
    !pip install git+https://github.com/goodchemistryco/Tangelo.git@develop --quiet
    !pip install qulacs pyscf --quiet
try:
    import mendeleev 
except ModuleNotFoundError:
    !pip install mendeleev --quiet

# Import for a pretty jupyter notebook.
import json

# Molecule definition.
from tangelo import SecondQuantizedMolecule as SQMol

# The minimal import foRr DMET.
from tangelo.problem_decomposition import DMETProblemDecomposition
# Ability to change localization method.
from tangelo.problem_decomposition.dmet import Localization
# Use for VQE ressources estimation vs DMET.
from tangelo.algorithms import VQESolver
# Use for comparison.
from tangelo.algorithms import FCISolver
from mendeleev import element

In [2]:
import numpy as np
from tangelo.toolboxes.ansatz_generator.ansatz import Ansatz
from tangelo.toolboxes.qubit_mappings.statevector_mapping import get_reference_circuit
from tangelo.toolboxes.qubit_mappings.mapping_transform import get_qubit_number, fermion_to_qubit_mapping
from tangelo.linq import Circuit, Gate

# Table Maker

## Simulate by basis

In [3]:
basis_sets = [
    "STO-3G",
    "3-21G",
    "6-31G",
    "minao",
    "LANL2DZ"
]

In [4]:
import time, math
import gc

#Circuit Complexity Function
def VQE_Complexity(vqe_mol):
    resources_mol_vqe = vqe_mol.get_resources()
    print(resources_mol_vqe)
    
    return resources_mol_vqe

# Open or create a file named "atom_table.txt"
with open("atom_table.txt", "a") as file:
    for basis_iteration in basis_sets:
        print(basis_iteration)
        
        sample_geometry ="""
        H   0. 0. 0.
        Li  0. 0. 1.5949
        """
        sample_basis = basis_iteration
        sample_charge = 0
        sample_symmetry = True
        sample_spin = 0
        sample_fo = 1
        
        mol = SQMol(sample_geometry, q=sample_charge, spin=sample_spin, symmetry = sample_symmetry, basis=sample_basis, frozen_orbitals=sample_fo)
        
        options_mol_vqe = {"molecule": mol, "qubit_mapping": "jw", "verbose": False}
        vqe_mol = VQESolver(options_mol_vqe)
        vqe_mol.build()
        fci_solver = FCISolver(mol)
        
        try:
            result = VQE_Complexity(vqe_mol)
            #change this value if you want to compute complex cases.
            if result.get('qubit_hamiltonian_terms') < 1001: 
                
                start_f = time.time()
                energy_fci = fci_solver.simulate()
                end_f = time.time()
                time_f = end_f - start_f
                print(f"FCI energy:\t{energy_fci}")
        
                start_v = time.time()
                energy_vqe = vqe_mol.simulate()
                end_v = time.time()
                time_v = end_v - start_v
                print(f"VQE energy:\t{energy_vqe}")
        
                file.write(f"{mol.elements} | {mol.basis} | FCI energy: {energy_fci} | VQE energy: {energy_vqe} | t_FCI: {time_f:.4f}s | t_VQE: {time_v:.4f}s\n")
    
                # Calculate the percentage difference in time and energy
                time_percentage = abs(time_f - time_v) / time_f * 100  # Percentage of VQE time compared to FCI
                accuracy_percentage = 1 - abs(energy_fci - energy_vqe) / abs(energy_fci) * 100  # Percentage of accuracy difference
    
                file.write(f"VQE took {time_percentage:.2f}% of the time compared to FCI simulation with {accuracy_percentage:.6f}% accuracy\n")
        except FileNotFoundError:
            file.write(f"Error: No File!\n\n")
        # Explicitly delete large objects
        del mol
        del vqe_mol
        del fci_solver
            
        # Run garbage collection
        gc.collect()

print("Information has been recorded in 'atom_table.txt'.")

STO-3G
{'qubit_hamiltonian_terms': 276, 'circuit_width': 10, 'circuit_depth': 796, 'circuit_2qubit_gates': 608, 'circuit_var_gates': 64, 'vqe_variational_parameters': 14}
FCI energy:	-7.882175990801267
VQE energy:	-7.882174843685822
3-21G
{'qubit_hamiltonian_terms': 4427, 'circuit_width': 20, 'circuit_depth': 5884, 'circuit_2qubit_gates': 4992, 'circuit_var_gates': 300, 'vqe_variational_parameters': 54}
6-31G
{'qubit_hamiltonian_terms': 4427, 'circuit_width': 20, 'circuit_depth': 5884, 'circuit_2qubit_gates': 4992, 'circuit_var_gates': 300, 'vqe_variational_parameters': 54}
minao
{'qubit_hamiltonian_terms': 27, 'circuit_width': 4, 'circuit_depth': 100, 'circuit_2qubit_gates': 64, 'circuit_var_gates': 12, 'vqe_variational_parameters': 2}
FCI energy:	-7.979873194994736
VQE energy:	-7.9798728861543555
LANL2DZ
{'qubit_hamiltonian_terms': 4427, 'circuit_width': 20, 'circuit_depth': 5884, 'circuit_2qubit_gates': 4992, 'circuit_var_gates': 300, 'vqe_variational_parameters': 54}
Information ha

## Simulate single time

In [5]:
import time, math
import gc

#Circuit Complexity Function
def VQE_Complexity(vqe_mol):
    resources_mol_vqe = vqe_mol.get_resources()
    print(resources_mol_vqe)
    
    return resources_mol_vqe

# Open or create a file named "atom_table.txt"
with open("atom_table.txt", "a") as file:
 
    sample_geometry ="""
    H   0. 0. 0.
    Li  0. 0. 1.5949
    """
    sample_basis = "minao"
    sample_charge = 0
    sample_symmetry = True
    sample_spin = 0
    sample_fo = 1
        
    mol = SQMol(sample_geometry, q=sample_charge, spin=sample_spin, symmetry = sample_symmetry, basis=sample_basis, frozen_orbitals=sample_fo)
        
    options_mol_vqe = {"molecule": mol, "qubit_mapping": "jw", "verbose": False}
    vqe_mol = VQESolver(options_mol_vqe)
    vqe_mol.build()
    fci_solver = FCISolver(mol)
        
    try:
        result = VQE_Complexity(vqe_mol)
        #change this value if you want to compute complex cases.
        if result.get('qubit_hamiltonian_terms') < 1001: 
            start_f = time.time()
            energy_fci = fci_solver.simulate()
            end_f = time.time()
            time_f = end_f - start_f
            print(f"FCI energy:\t{energy_fci}")
        
            start_v = time.time()
            energy_vqe = vqe_mol.simulate()
            end_v = time.time()
            time_v = end_v - start_v
            print(f"VQE energy:\t{energy_vqe}")
        
            file.write(f"{mol.elements} | {mol.basis} | FCI energy: {energy_fci} | VQE energy: {energy_vqe} | t_FCI: {time_f:.4f}s | t_VQE: {time_v:.4f}s\n")
    
            # Calculate the percentage difference in time and energy
            time_percentage = abs(time_f - time_v) / time_f * 100  # Percentage of VQE time compared to FCI
            accuracy_percentage = 1 - abs(energy_fci - energy_vqe) / abs(energy_fci) * 100  # Percentage of accuracy difference

            file.write(f"VQE took {time_percentage:.2f}% of the time compared to FCI simulation with {accuracy_percentage:.6f}% accuracy\n")
    except FileNotFoundError:
        file.write(f"Error: No File!\n\n")
    # Explicitly delete large objects
    del mol
    del vqe_mol
    del fci_solver
            
    # Run garbage collection
    gc.collect()

print("Information has been recorded in 'atom_table.txt'.")

{'qubit_hamiltonian_terms': 27, 'circuit_width': 4, 'circuit_depth': 100, 'circuit_2qubit_gates': 64, 'circuit_var_gates': 12, 'vqe_variational_parameters': 2}
FCI energy:	-7.9798731949947355
VQE energy:	-7.97987288615436
Information has been recorded in 'atom_table.txt'.


# Atom Orbital Checker:

In [6]:
# Dictionary to expand noble gas core configurations
noble_gas_configurations = {
    "[He]": "1s2",
    "[Ne]": "1s2 2s2 2p6",
    "[Ar]": "1s2 2s2 2p6 3s2 3p6",
    "[Kr]": "1s2 2s2 2p6 3s2 3p6 3d10 4s2 4p6",
    "[Xe]": "1s2 2s2 2p6 3s2 3p6 3d10 4s2 4p6 4d10 5s2 5p6",
    "[Rn]": "1s2 2s2 2p6 3s2 3p6 3d10 4s2 4p6 4d10 5s2 5p6 4f14 5d10 6s2 6p6"
}

def expand_electron_configuration(econf):
    # Split the electron configuration to handle noble gas
    parts = econf.split()
    expanded_config = []
    
    # Replace noble gas with full configuration
    for part in parts:
        if part in noble_gas_configurations:
            expanded_config.append(noble_gas_configurations[part])
        else:
            expanded_config.append(part)
    
    # Return the fully expanded configuration
    return " ".join(expanded_config)

In [7]:
# Example usage
element_symbol = "Cr"
atom = element(element_symbol)

# Get the electron configuration (shorthand)
shorthand_econf = atom.econf

# Expand the configuration
expanded_econf = expand_electron_configuration(shorthand_econf)

print(f"Element: {atom.name} ({atom.symbol})")
print(f"Expanded Electron Configuration: {expanded_econf}")
print(f"Atomic Number: {atom.atomic_number}")

Element: Chromium (Cr)
Expanded Electron Configuration: 1s2 2s2 2p6 3s2 3p6 3d5 4s
Atomic Number: 24
